This notebook contains the visual representation of creating the other tables and expanding some existing tables.  Uses FiveThirtyEight's common names dataset to randomly generate names.

Datasets: 

(1) https://www.kaggle.com/zynicide/nfl-football-player-stats  
(2) https://www.kaggle.com/fivethirtyeight/fivethirtyeight-most-common-name-dataset

In [1]:
import pickle
import pandas as pd
import numpy as np

from random import sample
from random import shuffle

## Re-open Old Pickles

In [2]:
with open(r'../Data/Pickles/offense.pickle', 'rb') as f:
    offense = pickle.load(f)

In [3]:
with open(r'../Data/Pickles/defense.pickle', 'rb') as f:
    defense = pickle.load(f)

In [4]:
offense.head()

,gameNumber,passYards,rushYards,recYards,interceptions,touchdowns,playerID,position
4,1,0,0,49,0,0,Emmanuel Sanders,WR
5,2,0,0,39,0,0,Emmanuel Sanders,WR
6,3,0,0,117,0,2,Emmanuel Sanders,WR
7,4,0,0,88,0,1,Emmanuel Sanders,WR
8,5,0,0,80,0,0,Emmanuel Sanders,WR


In [5]:
defense.head()

,playerID,gameNumber,sacks,interceptions,touchdowns,safeties,blocks
0,Arizona Cardinals,1,2.0,0,0,0,0
1,Arizona Cardinals,2,2.0,2,0,0,0
2,Arizona Cardinals,3,2.0,1,0,1,0
3,Arizona Cardinals,4,2.0,0,0,0,0
4,Arizona Cardinals,5,2.0,0,0,0,0


## Create users

In [6]:
names = pd.read_csv(r'../Data/Datasets/names.csv')

In [7]:
names = names[['FirstName', 'Surname']]
fnames = list(np.unique(names['FirstName'].to_numpy()))
lnames = list(np.unique(names['Surname'].to_numpy()))

In [8]:
print(f'{len(fnames)} first names')
print(f'{len(lnames)} last names')

20 first names
20 last names


To generate 20 names, let's sample 4 last names and 5 first names

In [9]:
sample_fnames = sample(fnames, 5)
sample_lnames = sample(lnames, 4)

list(zip(sample_fnames, sample_lnames))

[('David', 'Lee'),
 ('Daniel', 'Johnson'),
 ('Jennifer', 'Taylor'),
 ('Barbara', 'Garcia')]

In [10]:
generated_names = []
for f in sample_fnames:
    for l in sample_lnames:
        generated_names.append(f + ' ' + l)

In [11]:
print(generated_names)

['David Lee', 'David Johnson', 'David Taylor', 'David Garcia', 'Daniel Lee', 'Daniel Johnson', 'Daniel Taylor', 'Daniel Garcia', 'Jennifer Lee', 'Jennifer Johnson', 'Jennifer Taylor', 'Jennifer Garcia', 'Barbara Lee', 'Barbara Johnson', 'Barbara Taylor', 'Barbara Garcia', 'Charles Lee', 'Charles Johnson', 'Charles Taylor', 'Charles Garcia']


In [12]:
shuffle(generated_names)
print(generated_names)

['Jennifer Taylor', 'Barbara Garcia', 'David Lee', 'Daniel Lee', 'David Taylor', 'Daniel Taylor', 'Barbara Johnson', 'Charles Garcia', 'David Johnson', 'Charles Taylor', 'Jennifer Johnson', 'Daniel Johnson', 'Jennifer Garcia', 'Barbara Lee', 'Barbara Taylor', 'Charles Lee', 'Daniel Garcia', 'David Garcia', 'Charles Johnson', 'Jennifer Lee']


## Make "league" Table

In [13]:
league1 = generated_names[:10]
league2 = generated_names[10:]

In [14]:
league = pd.DataFrame([[1, 'family'], [2, 'competitive']], columns=['leagueID', 'leagueType'])
league.set_index('leagueID')

,leagueType
leagueID,
1,family
2,competitive


In [35]:
league.to_csv(r'../Data/CSV/leagues.csv', header=False, index=False)

## Sample players to help form contracts

In [16]:
l1_qbs = sample(list(offense[offense['position'] == 'QB']['playerID'].unique()), len(league1))
l1_rbs = sample(list(offense[offense['position'] == 'RB']['playerID'].unique()), 2*len(league1))
l1_wrs = sample(list(offense[offense['position'] == 'WR']['playerID'].unique()), 2*len(league1))
l1_defense = sample(list(defense['playerID'].unique()), len(league1))

In [17]:
l2_qbs = sample(list(offense[offense['position'] == 'QB']['playerID'].unique()), len(league2))
l2_rbs = sample(list(offense[offense['position'] == 'RB']['playerID'].unique()), 2*len(league2))
l2_wrs = sample(list(offense[offense['position'] == 'WR']['playerID'].unique()), 2*len(league2))
l2_defense = sample(list(defense['playerID'].unique()), len(league2))

## Make "contract" table

In [18]:
contract = pd.DataFrame(l1_qbs+l1_rbs+l1_wrs+l1_defense+l2_qbs+l2_rbs+l2_wrs+l2_defense, columns=['playerID'])
contract['owner'] = league1 + 2*league1 + 2*league1 + league1 + league2 + 2*league2 + 2*league2 + league2
contract['startWeek'] = 1
contract['endWeek'] = 16
contract['isActive'] = 1

In [19]:
def name_to_id(name):
    return name.split(' ')[1].lower()+name.split(' ')[0].lower()[:3]

In [20]:
contract['owner'] = contract['owner'].apply(name_to_id)

In [21]:
contract

,playerID,owner,startWeek,endWeek,isActive
0,Josh McCown,taylorjen,1,16,1
1,Marcus Mariota,garciabar,1,16,1
2,Ryan Mallett,leedav,1,16,1
3,Blake Bortles,leedan,1,16,1
4,Nick Foles,taylordav,1,16,1
...,...,...,...,...,...
115,Baltimore Ravens,leecha,1,16,1
116,Miami Dolphins,garciadan,1,16,1
117,New York Jets,garciadav,1,16,1
118,Jacksonville Jaguars,johnsoncha,1,16,1


## Create "user" table

In [22]:
user = pd.DataFrame([(name.split(' ')[0], name.split(' ')[1], name.split(' ')[1].lower()+name.split(' ')[0].lower()[:3]) 
                     for name in generated_names], 
                    columns=['nameFirst', 'nameLast', 'userID'])

In [23]:
user.head()

,nameFirst,nameLast,userID
0,Jennifer,Taylor,taylorjen
1,Barbara,Garcia,garciabar
2,David,Lee,leedav
3,Daniel,Lee,leedan
4,David,Taylor,taylordav


In [24]:
user.to_csv(r'../Data/CSV/users.csv', index=False, header=False)

## Create "team" table

In [25]:
leagues1 = {name: 1 for name in league1}
leagues2 = {name: 2 for name in league2}

leagues1.update(leagues2)

In [26]:
teams = pd.DataFrame([(f'Team{number}', name.split(' ')[1].lower()+name.split(' ')[0].lower()[:3], leagues1[name]) 
                      for number, name in enumerate(generated_names)],
                     columns=['teamID', 'userID', 'leagueID'])

In [27]:
teams

,teamID,userID,leagueID
0,Team0,taylorjen,1
1,Team1,garciabar,1
2,Team2,leedav,1
3,Team3,leedan,1
4,Team4,taylordav,1
5,Team5,taylordan,1
6,Team6,johnsonbar,1
7,Team7,garciacha,1
8,Team8,johnsondav,1
9,Team9,taylorcha,1


In [28]:
teams.to_csv(r'../Data/CSV/team.csv', index=False, header=False)

## Merge teams and contracts and drop names

In [29]:
real_contracts = pd.merge(contract, teams.rename({'userID': 'owner'}, axis=1), on='owner').drop(['owner', 'leagueID'], axis=1)

In [30]:
real_contracts

,playerID,startWeek,endWeek,isActive,teamID,leagueID
0,Josh McCown,1,16,1,Team0,1
1,Zach Zenner,1,16,1,Team0,1
2,Jacquizz Rodgers,1,16,1,Team0,1
3,Golden Tate,1,16,1,Team0,1
4,Russell Shepard,1,16,1,Team0,1
...,...,...,...,...,...,...
115,Giovani Bernard,1,16,1,Team19,2
116,Adrian Peterson,1,16,1,Team19,2
117,Cobi Hamilton,1,16,1,Team19,2
118,Vince Mayle,1,16,1,Team19,2


In [36]:
real_contracts.to_csv(r'../Data/CSV/contracts.csv', index=False, header=False)